# December 03, 2021

https://adventofcode.com/2021/day/3

In [1]:
import pandas as pd
import numpy as np

# Part 1

In [2]:
# read as string to get leading 0s
df = pd.read_csv("../data/2021/03.txt", dtype=str, names=["diag"])

In [3]:
# Parse individual digits
dignames = [str(x) for x in range( len(df["diag"][0]) )[::-1] ]
df[dignames] = df.apply(lambda x: [int(d) for d in x["diag"]], axis=1, result_type="expand")

In [4]:
# We could take a shortcut here using the fact that it's binary, but let's be more general
# get digit counts
digc = df[dignames].apply( lambda x: x.value_counts() )

In [5]:
gamma = pd.DataFrame(digc.apply(lambda x: x.idxmax(), axis=0), columns=["digit"])
gamma["pow"] = gamma.index
gamma["val"] = gamma.apply( lambda x: 2**int(x["pow"]) * x["digit"], axis=1)
gamma

,digit,pow,val
11,1,11,2048
10,0,10,0
9,0,9,0
8,0,8,0
7,1,7,128
6,1,6,64
5,1,5,32
4,0,4,0
3,0,3,0
2,1,2,4


In [6]:
epsilon = pd.DataFrame(digc.apply(lambda x: x.idxmin(), axis=0), columns=["digit"])
epsilon["pow"] = epsilon.index
epsilon["val"] = epsilon.apply( lambda x: 2**int(x["pow"]) * x["digit"], axis=1 )
epsilon

,digit,pow,val
11,0,11,0
10,1,10,1024
9,1,9,512
8,1,8,256
7,0,7,0
6,0,6,0
5,0,5,0
4,1,4,16
3,1,3,8
2,0,2,0


In [7]:
gam = gamma["val"].sum()
eps = epsilon["val"].sum()
print(gam, eps, gam*eps)

2277 1818 4139586


# Part 2

In [8]:
def select_max(x):
    vc = x.value_counts()
    # only one value present, return it
    if len(vc) == 1:
        return vc.index[0]
    
    if vc[0] > vc[1]:
        return 0
    else:
        return 1
    
def select_min(x):
    vc = x.value_counts()
    # if only one value present, return the other
    if len(vc) == 1:
        return 1 - vc.index[0]
    
    if vc[0] <= vc[1]:
        return 0
    else:
        return 1
    
def select_bit(x, which):
    if which == "min":
        return select_min(x)
    else:
        return select_max(x)
    
def filter_values( df, pos, which ):
    targ = select_bit( df[pos], which )

    return df.loc[ df[pos] == targ ]

def get_rating( df, which, debug = 0 ):
    cols = [x for x in df.columns if x != "diag"]
    cidx = 0

    rcnt = df.shape[0]
    while rcnt > 1 and cidx < len(cols):
        c = cols[cidx]
        print("Checking col", c)
        df = filter_values( df, cols[cidx], which )
        rcnt = df.shape[0]
        if debug >= 1:
            print("rows left:", rcnt)
        if debug == 2:
            display(df)

        cidx += 1
    assert rcnt > 0, "All values eliminated!"
    assert rcnt == 1, "Multiple values remaining!"

    return df

def read_val( df ):
    cols = [x for x in df.columns if x != "diag"]
    vals = [ 2**int(c) * df[c].iloc[0] for c in cols ]
    return sum(vals)

In [9]:
o2 = get_rating(df, "max", debug=0)
print(o2, "\n----------\n")
co2 = get_rating(df, "min", debug=0)
print(co2)

Checking col 11
Checking col 10
Checking col 9
Checking col 8
Checking col 7
Checking col 6
Checking col 5
Checking col 4
Checking col 3
Checking col 2
Checking col 1
Checking col 0
             diag  11  10  9  8  7  6  5  4  3  2  1  0
805  100111101011   1   0  0  1  1  1  1  0  1  0  1  1 
----------

Checking col 11
Checking col 10
Checking col 9
Checking col 8
Checking col 7
Checking col 6
Checking col 5
Checking col 4
Checking col 3
             diag  11  10  9  8  7  6  5  4  3  2  1  0
326  001011000101   0   0  1  0  1  1  0  0  0  1  0  1


In [10]:
o2_rating = read_val(o2)
co2_rating = read_val(co2)
print( o2_rating, co2_rating, o2_rating*co2_rating )

2539 709 1800151


In [11]:
display(o2)
display(co2)

,diag,11,10,9,8,7,6,5,4,3,2,1,0
805,100111101011,1,0,0,1,1,1,1,0,1,0,1,1


,diag,11,10,9,8,7,6,5,4,3,2,1,0
326,001011000101,0,0,1,0,1,1,0,0,0,1,0,1


In [12]:
512+128+64+4+1

709